Quantum State Preparation - QSP

In [54]:
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit.quantum_info import Statevector

In [86]:
def preparacao_de_estados(n):
    if n % 2 != 0:
        raise ValueError("O número de qubits deve ser par.")
    
    qc = QuantumCircuit(n)

    k = n//2
    
    # Phase 1
    
    n_cnots = (2**k) - k - 1 # Número de C-NOTs
    contador = 0             # Conta o número de C-NOTs usados

    # Loop para gerar todas as combinações possíveis de controle e alvo
    while contador < n_cnots:
        for i in range(k):  # Controle
            for j in range(i + 1, k):  # Alvo (sempre maior que o controle)
                qc.cx(i, j)  # Adiciona o par (controle, alvo)
                contador += 1
                if contador == n_cnots:  # Se atingir o número exato, sai do loop
                    break
            if contador == n_cnots:
                break

    # Phase 2

    for i in range(k):
        qc.cx(i, i+k)
    
    # Phase 3 & 4

    for i in range(k):
        qc.cx((i*2) + 1, i*2)
        qc.cx(i*2, (i*2) + 1)
        qc.cx((i*2) + 1, i*2)

    return qc

# Exemplo de uso
n = 2
qc = preparacao_de_estados(n)
qc.draw()

┌───┐     ┌───┐
q_0: ──■──┤ X ├──■──┤ X ├
     ┌─┴─┐└─┬─┘┌─┴─┐└─┬─┘
q_1: ┤ X ├──■──┤ X ├──■──
     └───┘     └───┘

In [57]:
qc.measure_all()

# initialization of the sampler
sampler = Sampler()

job = sampler.run([qc])
result = job.result()
print(f" > Counts: {result[0].data.meas.get_counts()}")

 > Counts: {'0000': 1024}


In [41]:
sv = Statevector.from_label("0+")
sv.probabilities_dict()

{np.str_('00'): np.float64(0.4999999999999999),
 np.str_('01'): np.float64(0.4999999999999999)}

Quantum Error Correction - QEC